In [1]:
import sys
sys.path.append('/home/potzschf/repos/')
from helperToolz.helpsters import *
from helperToolz.dicts_and_lists import *
from helperToolz.guzinski import * 
from other_repos.pyDMS.pyDMS.pyDMS import *
import shutil
import time

In [ ]:
x = 70
y = 44
year = 2019
bands = [file for file in getFilelist(f'/data/Aldhani/eoagritwin/force/output/Guzinski/{year}/tiles/X00{x}_Y00{y}/', '.tif') if '_20190617' in file]
bands = [bands[i] for i in [0,1,6,3,4,5,2,7,8]]

In [ ]:
ds = gdal.Open(f'/data/Aldhani/eoagritwin/et/Auxiliary/DEM/Force_Tiles/SLOPE/SLOPE_X00{x}_Y00{y}.tif')
slope = ds.GetRasterBand(1).ReadAsArray()
ds = gdal.Open(f'/data/Aldhani/eoagritwin/et/Auxiliary/DEM/Force_Tiles/ASPECT/ASPECT_X00{x}_Y00{y}.tif')
aspect = ds.GetRasterBand(1).ReadAsArray()

In [ ]:
arrs = []
for band in bands:
    ds = gdal.Open(band)
    arrs.append(ds.GetRasterBand(1).ReadAsArray())
arrs.append(slope)
arrs.append(aspect)
arr = np.dstack(arrs)
mask = np.logical_and(arr[:,:,9] < 0.000000001, arr[:,:,10] == 180)
masked_arr = np.where(mask[:,:,None],np.nan, arr)
npTOdisk(masked_arr, reference_path=bands[0], outPath='/data/Aldhani/eoagritwin/et/Auxiliary/trash/S2_stack_test.tif', bands=len(bands))

In [ ]:
highResFilename = '/data/Aldhani/eoagritwin/et/Auxiliary/trash/S2_stack_test.tif'
lowResFilename = '/data/Aldhani/eoagritwin/et/Auxiliary/trash/LST.tif'
lowResMaskFilename = r""
outputFilename = '/data/Aldhani/eoagritwin/et/Auxiliary/trash/sharpi_test.tif'

##########################################################################################

if __name__ == "__main__":

    useDecisionTree = True

    commonOpts = {"highResFiles":               [highResFilename],
                  "lowResFiles":                [lowResFilename],
                  "lowResQualityFiles":         [lowResMaskFilename],
                  "lowResGoodQualityFlags":     [255],
                  "cvHomogeneityThreshold":     0,
                  "movingWindowSize":           15,
                  "disaggregatingTemperature":  True}
    dtOpts =     {"perLeafLinearRegression":    True,
                  "linearRegressionExtrapolationRatio": 0.25}
    sknnOpts =   {'hidden_layer_sizes':         (10,),
                  'activation':                 'tanh'}
    nnOpts =     {"regressionType":             REG_sklearn_ann,
                  "regressorOpt":               sknnOpts}

    start_time = time.time()

    if useDecisionTree:
        opts = commonOpts.copy()
        opts.update(dtOpts)
        disaggregator = DecisionTreeSharpener(**opts)
    else:
        opts = commonOpts.copy()
        opts.update(nnOpts)
        disaggregator = NeuralNetworkSharpener(**opts)

    print("Training regressor...")
    disaggregator.trainSharpener()
    print("Sharpening...")
    downscaledFile = disaggregator.applySharpener(highResFilename, lowResFilename)
    print("Residual analysis...")
    residualImage, correctedImage = disaggregator.residualAnalysis(downscaledFile, lowResFilename,
                                                                   lowResMaskFilename,
                                                                   doCorrection=True)
    print("Saving output...")
    highResFile = gdal.Open(highResFilename)
    if correctedImage is not None:
        outImage = correctedImage
    else:
        outImage = downscaledFile
    # outData = utils.binomialSmoother(outData)
    outFile = utils.saveImg(outImage.GetRasterBand(1).ReadAsArray(),
                            outImage.GetGeoTransform(),
                            outImage.GetProjection(),
                            outputFilename)
    residualFile = utils.saveImg(residualImage.GetRasterBand(1).ReadAsArray(),
                                 residualImage.GetGeoTransform(),
                                 residualImage.GetProjection(),
                                 os.path.splitext(outputFilename)[0] + "_residual" +
                                 os.path.splitext(outputFilename)[1])

    outFile = None
    residualFile = None
    downsaceldFile = None
    highResFile = None

    print(time.time() - start_time, "seconds")

In [ ]:
# rename super long folders
for fo in ['S2only', 'allpred']:
    dirs = [file for file in getFilelist(f'/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/sharpened2/{fo}/', 'tif', deep=True) if 'resid' not in file]

    long_names = []
    for dir in dirs:
        long_names.append(dir.split('/')[10])

    long_names = list(set(long_names))

    for cdx, long in enumerate(long_names):
        dir_subs = [dir for dir in dirs if long in dir]
        for dir_sub in dir_subs:
            start = '/'.join(dir_sub.split('/')[0:9])
            end = '/'.join(dir_sub.split('/')[11:])
            dest_dir = f'{start}/{fo}short/t{cdx}/{end}'
            os.makedirs(os.path.dirname(dest_dir), exist_ok=True)
            shutil.copy2(dir_sub, dest_dir)

In [2]:
## make vrts from single FORCE tile tests

base_path = '/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/sharpened2/'

folders = ['allpred', 'S2only']

comps = ['maxLST', 'minVZA']
days = [f'July_{i:02d}' for i in range(19,28,1)]
#days = [f'May_{i:02d}' for i in range(8,17,1)]

masks = ['S2Masked_withLSTmask', 'S2Masked_withoutLSTmask', 'S2notMasked_withLSTmask', 'S2notMasked_withoutLSTmask']

for path in [f'{base_path}vrt/{day}' for day in days]:
    os.makedirs(path, exist_ok=True)

In [4]:
for folder in folders:
    for comp in comps:
        for day in days:
            for mask in masks:
                file_list = [file for file in getFilelist(base_path + folder, '.tif', deep=True) if comp in file and day in file and mask in file and 'resid' not in file]
                outpath = base_path + f'vrt/{day}/{folder}_{comp}_{day}_{mask}.vrt'
                gdal.BuildVRT(outpath, file_list, separate=False)
                convertVRTpathsTOrelative(outpath)

In [ ]:
# export statistics

results = []

for folder in folders:
    for comp in comps:
        for day in days:
            for mask in masks:
                file_list = [file for file in getFilelist(base_path + folder, '.tif', deep=True) if comp in file and day in file and mask in file and 'resid' not in file]
                outpath = base_path + f'vrts/{day}/{folder}_{comp}_{day}_{mask}.vrt'

                # open dataset
                ds = gdal.Open(outpath)
                arr = ds.GetRasterBand(1).ReadAsArray()
                
                maxLST = np.nanmax(arr)
                minLST = np.nanmin(arr)
                p25, p50, p75 = np.nanpercentile(arr, [25, 50, 75])

                # Append result
                results.append({
                    'folder': folder,
                    'comp': comp,
                    'day': day,
                    'mask': mask,
                    'maxLST': maxLST,
                    'minLST': minLST,
                    '25thLST': p25,
                    'medianLST': p50,
                    '75thLST': p75
                })


# convert and export
df = pd.DataFrame(results)
df.to_csv('../csvs/basic_stats_vrt_44.csv', index=False)


In [30]:
# make evap vrt
base_path = '/data/Aldhani/eoagritwin/et/S2_ETa/44/'
file_list = [file for file in getFilelist(f'{base_path}single_tiles/', 'tif') if 'July_09' in file]
outpath = f'{base_path}vrts/{'_'.join(file_list[0].split('/')[-1].split('_')[:-1])}.vrt'
gdal.BuildVRT(outpath, file_list, separate=False)
convertVRTpathsTOrelative(outpath)